In [1]:
# import dependencies
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg19 import (
    VGG19,
    preprocess_input,
    decode_predictions
)

Using TensorFlow backend.


In [2]:
names = ['with_mask', 'without_mask']
batch = 32

In [3]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True,
        validation_split = 0.2)
train_datagen

In [4]:
train_generator = train_datagen.flow_from_directory(
    directory = 'Data/',
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical',
    subset = 'training')

Found 8834 images belonging to 2 classes.


In [5]:
test_generator = train_datagen.flow_from_directory(
    directory = 'Data/',
    target_size = (220,220),
    batch_size = batch,
    class_mode = 'categorical',
    subset = 'validation')

Found 2208 images belonging to 2 classes.


In [6]:
from keras import Sequential
from keras.layers import Flatten, Dense

In [7]:
vgg19 = VGG19(weights = "imagenet", include_top = False, input_shape = (220,220,3))

for layers in vgg19.layers:
    layers.trainable = False

In [8]:
model = Sequential()
model.add(vgg19)
model.add(Flatten())
model.add(Dense(units = 2, activation = 'sigmoid'))

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 6, 6, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 36866     
Total params: 20,061,250
Trainable params: 36,866
Non-trainable params: 20,024,384
_________________________________________________________________


In [10]:
model.compile(optimizer = 'Adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [14]:
model.fit(train_generator,
          steps_per_epoch = len(train_generator)//32,
          epochs = 10,
          validation_data = test_generator,
          validation_steps=len(test_generator)//32,
          verbose = 1
         )

Epoch 1/10
3/8 [==========>...................] - ETA: 8:33 - loss: 0.4265 - accuracy: 0.8542 

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001664D7C0B48>